# EfficientNet Experiments

### Load packages (and magic formulas), define model, and download datasets

In [1]:
%load_ext autoreload
import timm
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet34
from torch import nn
import torch.optim as optim
from torch import nn
import copy
import pruning_funcs
import numpy as np
import matplotlib.pyplot as plt

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# Load CIFAR-10
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


### Load Pre-trained EfficientNet weights

In [2]:
# Load pre-trained EfficientNet
model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=10)
device = torch.device("mps")
model = model.to(device)

### Define hyperparameters for further training

In [3]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Train or Load Weights
This cell is where we either train the model (lines 1-11), or load our previously trained model (lines 13-14). The intention is to either do one or the other, not both.

In [4]:
# Training loop
# for epoch in range(10):  # number of epochs
#     model.train()
#     for images, labels in train_loader:
#         images, labels = images.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#     print(f'Epoch {epoch+1}, Loss: {loss.item()}')

state_dict = torch.load("efficientnet_state")
model.load_state_dict(state_dict)

<All keys matched successfully>

### Define method for evaluating accuracy

In [5]:
def evaluate_model(testing_model, dataloader, suppress_output=False):
    testing_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = testing_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    if not suppress_output:
        print(f'Accuracy of the model on the test images: {accuracy}%')
    return accuracy

# Pruning methods begin here

### Test normalized Laplace pruning method

In [6]:
# Normalized Laplace

%autoreload now

tests = np.arange(0.008, 0.081, 0.008)
num_trials = 5

print()
print('Avg. accuracy at scale values for Normalized Laplace Distribution')

print()
print('Unpruned Model')

accuracy = evaluate_model(model, test_loader, suppress_output=True)
percent_zero = pruning_funcs.percent_zero_weights(model)
print(f'Accuracy: {accuracy:.2f}%\tPercent Zero: {percent_zero:.2f}%')

print()
print("Pruned Models (Normalized Laplace)")
for prune_scale in tests:
    accuracy = 0.
    percent_zeros = 0.
    for i in range(num_trials):
        pruned_model = copy.deepcopy(model)
        pruning_funcs.normalized_laplace_prune(pruned_model, device, scale=prune_scale)
        accuracy += evaluate_model(pruned_model, test_loader, suppress_output=True)
        percent_zeros += pruning_funcs.percent_zero_weights(pruned_model)
    accuracy /= num_trials
    percent_zeros /= num_trials
    print(f'Scale: {prune_scale:.3f}\tAvg. Accuracy: {accuracy:.2f}%\tAvg. Percent Zero: {percent_zeros:.2f}%')


Avg. accuracy at scale values for Normalized Laplace Distribution

Unpruned Model
Accuracy: 93.01%	Percent Zero: 0.00%

Pruned Models (Normalized Laplace)
Scale: 0.008	Avg. Accuracy: 92.97%	Avg. Percent Zero: 5.42%
Scale: 0.016	Avg. Accuracy: 92.90%	Avg. Percent Zero: 10.67%
Scale: 0.024	Avg. Accuracy: 92.23%	Avg. Percent Zero: 15.61%
Scale: 0.032	Avg. Accuracy: 91.38%	Avg. Percent Zero: 20.24%
Scale: 0.040	Avg. Accuracy: 89.68%	Avg. Percent Zero: 24.51%
Scale: 0.048	Avg. Accuracy: 81.53%	Avg. Percent Zero: 28.43%
Scale: 0.056	Avg. Accuracy: 63.17%	Avg. Percent Zero: 32.01%
Scale: 0.064	Avg. Accuracy: 50.05%	Avg. Percent Zero: 35.31%
Scale: 0.072	Avg. Accuracy: 24.65%	Avg. Percent Zero: 38.35%
Scale: 0.080	Avg. Accuracy: 11.62%	Avg. Percent Zero: 41.12%


### Test percent by layer pruning method

In [7]:
# Percent by Layer

%autoreload now

percents = np.arange(5,51,5)
print()
print("Pruned Models (Standard Percent Pruning By Layer)")
for percent in percents:
    pruned_model = copy.deepcopy(model)
    pruning_funcs.percent_prune_by_layer(pruned_model, device, percent=percent)
    accuracy = evaluate_model(pruned_model, test_loader, suppress_output=True)
    percent_zero = pruning_funcs.percent_zero_weights(pruned_model)
    print(f'Theoretic percent pruned: {percent}%\tActual percent pruned: {percent_zero:.2f}%\tAccuracy: {accuracy:.2f}%')


Pruned Models (Standard Percent Pruning By Layer)
Theoretic percent pruned: 5%	Actual percent pruned: 5.00%	Accuracy: 82.65%
Theoretic percent pruned: 10%	Actual percent pruned: 10.00%	Accuracy: 26.52%
Theoretic percent pruned: 15%	Actual percent pruned: 15.00%	Accuracy: 10.16%
Theoretic percent pruned: 20%	Actual percent pruned: 20.00%	Accuracy: 10.00%
Theoretic percent pruned: 25%	Actual percent pruned: 25.00%	Accuracy: 10.00%
Theoretic percent pruned: 30%	Actual percent pruned: 30.00%	Accuracy: 10.00%
Theoretic percent pruned: 35%	Actual percent pruned: 35.00%	Accuracy: 10.00%
Theoretic percent pruned: 40%	Actual percent pruned: 40.00%	Accuracy: 10.00%
Theoretic percent pruned: 45%	Actual percent pruned: 45.00%	Accuracy: 10.00%
Theoretic percent pruned: 50%	Actual percent pruned: 50.00%	Accuracy: 10.00%


### Test bottom percent pruning method

In [8]:
# Bottom Percent

%autoreload now

percents = np.arange(5,51,5)
print()
print("Pruned Models (Standard Bottom Percent Pruning)")
for percent in percents:
    pruned_model = copy.deepcopy(model)
    pruning_funcs.bottom_percent_prune(pruned_model, device, percent=percent)
    accuracy = evaluate_model(pruned_model, test_loader, suppress_output=True)
    percent_zero = pruning_funcs.percent_zero_weights(pruned_model)
    print(f'Theoretic percent pruned: {percent}%\tActual percent pruned: {percent_zero:.2f}%\tAccuracy: {accuracy:.2f}%')



Pruned Models (Standard Bottom Percent Pruning)
Theoretic percent pruned: 5%	Actual percent pruned: 5.00%	Accuracy: 92.98%
Theoretic percent pruned: 10%	Actual percent pruned: 10.00%	Accuracy: 92.89%
Theoretic percent pruned: 15%	Actual percent pruned: 15.00%	Accuracy: 92.89%
Theoretic percent pruned: 20%	Actual percent pruned: 20.00%	Accuracy: 92.80%
Theoretic percent pruned: 25%	Actual percent pruned: 25.00%	Accuracy: 92.80%
Theoretic percent pruned: 30%	Actual percent pruned: 30.00%	Accuracy: 92.22%
Theoretic percent pruned: 35%	Actual percent pruned: 35.00%	Accuracy: 92.04%
Theoretic percent pruned: 40%	Actual percent pruned: 40.00%	Accuracy: 90.31%
Theoretic percent pruned: 45%	Actual percent pruned: 45.00%	Accuracy: 88.48%
Theoretic percent pruned: 50%	Actual percent pruned: 50.00%	Accuracy: 79.47%


### Test standard threshold (magnitude) based pruning method

In [9]:
# Threshold

%autoreload now

thresholds = np.arange(0.008, 0.081, 0.008)

print()
print("Pruned Models (Standard Threshold Pruning)")
for threshold in thresholds:
    pruned_model = copy.deepcopy(model)
    pruning_funcs.threshold_prune(pruned_model, threshold)
    accuracy = evaluate_model(pruned_model, test_loader, suppress_output=True)
    percent_zero = pruning_funcs.percent_zero_weights(pruned_model)
    print(f'Threshold: {threshold:.3f}\tPercent pruned: {percent_zero:.2f}%\tAccuracy: {accuracy:.2f}%')



Pruned Models (Standard Threshold Pruning)
Threshold: 0.008	Percent pruned: 5.44%	Accuracy: 92.94%
Threshold: 0.016	Percent pruned: 10.86%	Accuracy: 93.01%
Threshold: 0.024	Percent pruned: 16.23%	Accuracy: 92.73%
Threshold: 0.032	Percent pruned: 21.48%	Accuracy: 92.89%
Threshold: 0.040	Percent pruned: 26.59%	Accuracy: 92.53%
Threshold: 0.048	Percent pruned: 31.57%	Accuracy: 92.36%
Threshold: 0.056	Percent pruned: 36.39%	Accuracy: 91.68%
Threshold: 0.064	Percent pruned: 41.04%	Accuracy: 89.42%
Threshold: 0.072	Percent pruned: 45.48%	Accuracy: 88.34%
Threshold: 0.080	Percent pruned: 49.74%	Accuracy: 80.94%
